In [2]:
#end to end LLM project using pinecone vector database

import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI



In [3]:
from dotenv import load_dotenv
load_dotenv()

import os

In [4]:
#lets read document

def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [6]:
doc=read_doc('document/')

In [7]:
len(doc)

358

In [14]:
##divide the doc into chunks

def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [15]:
documents=chunk_data(docs=doc)
documents


[Document(page_content=' \nPradhan Mantri Fasal Bima Yojana Brief Guidelines and FAQ’s  \nWhat is the objective of implementation of PMFBY scheme? \nPMFBY is a risk mitigation tools aims at providing financial support and stabilizing the income of farmers \nto ensure their continuance in farming. It covers the perils to the crops arising out of unforeseen events \nat all stages, i.e. from sowing to post harvest,. It also encourages the farmers to adopt the modern and \ninnovative agriculture practices in order to have a stabilize income and sustainable production in \nagriculture sector.  \nWho are eligibly covered under this scheme? \nAll Farmer having insurable interest can be covered under these scheme including sharecroppers and \ntenant farmers. Further, Covered farmer are divided under 2 components:- \na. Compulsory Components: -  Those farmers who are availing Seasonal Agricultural Operations \nLoan from the Financial Institutions would be covered compulsorily i.e. Loanee Farmer

In [20]:
#embeddings technique

embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000021EF9D4B6D0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000021EF9DF3610>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-rugE0mmuVS8D1V50rmXnT3BlbkFJ0Ef3DTlGqLOW07V8KTFH', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [21]:
vectors=embeddings.embed_query('how are you?')
len(vectors)

1536

In [53]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'b097ca9f-3840-4866-b012-a529dd045e3a'

# configure client
pc = Pinecone(api_key=api_key)


In [56]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [57]:
index_name = 'langchainvector'

In [69]:
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

vectorstore.add_documents(doc)

ValueError: Pinecone API key must be provided in either `pinecone_api_key` or `PINECONE_API_KEY` environment variable